## 1. Define Classes

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cufflinks as cf
cf.go_offline(connected=True)
from scipy import optimize 
from datetime import datetime
from dateutil.relativedelta import relativedelta
%matplotlib inline

try:
  import pandas_datareader.data as web
  import chart_studio as py
except:
  !pip install pip pandas-datareader
  !pip install chart_studio
  import pandas_datareader.data as web
  import chart_studio as py

In [4]:
class Asset():
  def __init__(self, ticker, name, start, end):
    self.ticker = ticker
    self.name = name
    
    self.start = start
    self.end = end

    # 데이터 불러오기
    self.data = web.get_data_yahoo(self.ticker, self.start, self.end)
    self.date = self.data.index

    self.ratio = 0
    self.leverage = 0
    self.price = None
    self.change = None

  def put_ratio_leverage(self, ratio, leverage):
    self.ratio = ratio
    self.leverage = leverage

  def put_price_change(self):
    self.price = self.data['Adj Close']
    self.change = np.multiply(self.price.pct_change().to_list(), [100 * self.leverage])

  def get_date(self):
    return self.date

  def get_change(self):
    return self.change

  def get_name(self):
    return self.name

In [5]:
class GMV_Asset(Asset):
    def __init__(self, ticker_list, start, end):
        self.ticker_list = ticker_list
        self.start = start
        self.end = end
        self.df = pd.DataFrame(columns=self.ticker_list)
        self.non_zero_assets = [] 
        self.non_zero_weights = []
        self.asset_list = []
        
    def create_df(self):
        for stock in self.ticker_list:
            asset = Asset(stock, stock, self.start, self.end)
            self.asset_list.append(asset)
            self.df[stock] = asset.data["Adj Close"]

            if stock == self.ticker_list[0]:
                self.df['date'] = asset.data.index
                self.df.set_index('date', inplace=True)
            
    def get_returns(self):
        return self.df.pct_change()

    def get_mean_returns(self):
        return self.get_returns().mean()
    
    def get_cov_matrix(self):
        return self.get_returns().cov()
    
    def get_non_zero_weights(self, weights):
        for i in range(len(weights)):
            if weights[i] != 0:
                self.non_zero_weights.append(weights[i])
                self.non_zero_assets.append(self.asset_list[i])

In [7]:
class Portfolio():
  def __init__(self, name, assets, ratios, leverages, initial_balance, rebalancing_interval, gmv_weights=[]):
    self.name = name
    
    # list of class asset elements
    self.assets = assets

    # Assert all the dates for assets are equal and set portfolio start, end dates
    dates = [each.get_date() for each in self.assets]
    lendates = [len(each) for each in dates]
    assert len(set(lendates)) == 1
    self.date = dates[0]

    self.ratios = [each / sum(ratios) for each in ratios]
    self.leverages = leverages

    for i in range(len(self.assets)):
      self.assets[i].put_ratio_leverage(self.ratios[i], self.leverages[i])
      self.assets[i].put_price_change()

    self.initial_balance = initial_balance  
    self.rebalancing_interval = rebalancing_interval

    self.gmv_weights = []     # weights for gmv
    self.backtest_df = None
    self.backtest_result_df = None
    self.summary = None

  def backtest(self):
    # 첫 거래일
    balances = []
    
    asset_df = pd.DataFrame()
    for i in range(len(self.assets)):
      asset_df[i] = self.assets[i].data["Adj Close"].pct_change()
    print("EUM100: ", asset_df.head(50))
    
    for i in range(len(self.assets)):
      balance = [self.initial_balance * self.ratios[i]]
      balances.append(balance)
    
    total_balance = [self.initial_balance]
    next_rebalancing = [self.date[0] + relativedelta(months=self.rebalancing_interval)]

    # 이후 거래일
    for i in range(1, len(self.date)):
      total_balance_tmp = 0

      if self.date[i] >= next_rebalancing[i-1]: # 리밸런싱하는 날
        # 다음 리밸런싱 날짜 추가
        print("EUM0 현재 날짜 정보: ", self.date[i])
        next_rebalancing.append(next_rebalancing[i-1] + relativedelta(months=self.rebalancing_interval))
        
        if self.name == "GMV":
          GMV = GMVPortfolio(asset_df.mean(), asset_df.cov(), num_portfolios=25000, risk_free_rate=0.0178)
          self.gmv_weights = [round(i, 2) for i in GMV.min_variance().x]
          print("EUM2 현재 weight: ", self.gmv_weights)
        
          for j in range(len(self.assets)):
              print("EUM9 현재 주식 번호: ", j)
              balance = total_balance[i-1] * self.gmv_weights[j] * (1 + self.assets[j].get_change()[i] / 100)
              balances[j].append(balance)
              total_balance_tmp += balances[j][i]
            
              if j > 0:
#                 print("EUM1 이전 주식 가격: ", asset_df[j][i - 1])    
#                 print("EUM2 현재 주식 가격: ", asset_df[j][i])
                print("EUM3 주식 가격 차이: ", asset_df[j][i])
                print()
                print("EUM4 이전 투자 금액: ", balances[j][i -1])
                print("EUM5 현재 투자 금액: ", balances[j][i])
                
                ret = (balances[j][i]/balances[j][i-1]) - 1
                
                print("EUM5 투자 금액 차이: ", ret)
                print("EUM6 이상한 숫자   : ", 1 + self.assets[j].get_change()[i] / 100)
                print("--------------------------------------------------------------")
                print("EUM7 수익률 업데이트: ", asset_df[j][i], " -> ", ret)
                asset_df[j][i] = ret
        else:
          for j in range(len(self.assets)):
              balance = total_balance[i-1] * self.ratios[j] * (1 + self.assets[j].get_change()[i] / 100)
              balances[j].append(balance)
              total_balance_tmp += balances[j][i]
      else:
        # 이전 리밸런싱 날짜랑 동일하게
        next_rebalancing.append(next_rebalancing[i-1])
        for j in range(len(self.assets)):
          try:
            balances[j].append(balances[j][i-1] * (1 + self.assets[j].get_change()[i] / 100))
          except:
            balances[j].append(balances[j][i-1])
          total_balance_tmp += balances[j][i]
      total_balance.append(total_balance_tmp)

    df = pd.DataFrame()
    df['Date'] = self.date
    df.set_index('Date', inplace=True)
    df['Total'] = total_balance

    for i in range(len(self.assets)):
      df[self.assets[i].get_name()] = balances[i]
    
    print('Portfolio Backtest Complete')
    self.backtest_df = df

    return df

  def balance_result(self, balance):
    change = [0]
    cum_return = [0]
    prev_high = [0]
    prev_high_tmp = 0
    drawdown = [0]
    mdd = [0]
    mdd_tmp = 0
    CAGR = [0]
    stdev = [0]
    sharpe = [0]

    time_period = (self.date[-1] - self.date[0]).days / 365

    for i in range(1, len(self.date)):
      change.append((balance[i] / balance[i-1] - 1) * 100)
      
      cum_return.append((balance[i] / balance[0] - 1) * 100)

      if prev_high_tmp < cum_return[i]:
        prev_high_tmp = cum_return[i]
      else:
        prev_high_tmp = prev_high_tmp
      prev_high.append(prev_high_tmp)

      CAGR.append(((balance[i] / balance[0]) ** (1 / float(time_period)) - 1) * 100)

      drawdown.append(((cum_return[i] + 100) / (prev_high[i] + 100) - 1) * 100)

      if mdd_tmp > drawdown[i]:
        mdd_tmp = drawdown[i]
      else:
        mdd_tmp = mdd_tmp
      mdd.append(mdd_tmp)

      stdev.append(np.std(change))
      if stdev[i] != 0:
        sharpe.append(np.sqrt(252) * np.mean(change) / np.std(change))
      else:
        sharpe.append(0)

    return change, cum_return, prev_high, CAGR, drawdown, mdd, stdev, sharpe

  def backtest_result(self):
    df = pd.DataFrame()
    df['Date'] = self.date
    df.set_index('Date', inplace=True)
    label = ['Change', 'Cum. Return', 'Prev. High', 'CAGR', 'Drawdown', 'MDD', 'St. Dev', 'Sharpe']

    result = dict()
    for i in self.backtest_df.columns:
      result[i] = self.balance_result(self.backtest_df[i].to_list())
      df[f'{i} Balance'] = self.backtest_df[i].to_list()
      print(f'{i} Complete')
    for j in range(len(label)):
      for i in self.backtest_df.columns:
        df[f'{i} {label[j]}'] = result[i][j]

    self.backtest_result_df = df

    return df
  
  def periodic_result(self, mode):
    df = pd.DataFrame()

    for label in self.backtest_df.columns:
      return_points = []
      returns = []
      start_balance = []
      end_balance = []
      start = self.backtest_df[label].to_list()[0]
      
      if mode == 'annual':
        for i in range(1, len(self.date)):
          if self.date[i].year != self.date[i-1].year:
            return_points.append(self.date[i-1].year)
            returns.append((self.backtest_df[label].to_list()[i-1] / start - 1) * 100)
            start_balance.append(start)
            end_balance.append(self.backtest_df[label].to_list()[i-1])
            start = self.backtest_df[label].to_list()[i]
          elif self.date[i] == self.date[-1]: # 마지막 거래일
            return_points.append(self.date[i].year)
            returns.append((self.backtest_df[label].to_list()[i] / start - 1) * 100)
            start_balance.append(start)
            end_balance.append(self.backtest_df[label].to_list()[i-1])
        df[f'{label} {mode.capitalize()} Return'] = returns
        
      elif mode == 'monthly':
        for i in range(1, len(self.date)):
          if self.date[i].month != self.date[i-1].month:
            return_points.append(self.date[i-1].strftime('%Y-%m'))
            returns.append((self.backtest_df[label].to_list()[i-1] / start - 1) * 100)
            start_balance.append(start)
            end_balance.append(self.backtest_df[label].to_list()[i-1])
            start = self.backtest_df[label].to_list()[i]
          elif self.date[i] == self.date[-1]: # 마지막 거래일
            return_points.append(self.date[i].strftime('%Y-%m'))
            returns.append((self.backtest_df[label].to_list()[i] / start - 1) * 100)
            start_balance.append(start)
            end_balance.append(self.backtest_df[label].to_list()[i-1])
        df[f'{label} {mode.capitalize()} Return'] = returns

    df[f'Return {mode.capitalize()}'] = return_points
    df.set_index(f'Return {mode.capitalize()}', inplace=True)

    print(f'{mode.capitalize()} Result Complete')

    return df

  def get_name(self):
    return self.name

  def get_date(self):
    return self.date

  def get_backtest_result(self):
    return self.backtest_result_df

  def get_summary(self):
    # columns=['Detail', 'Initial Balance', 'Final Balance', 'CAGR', 'MDD', 'St. Dev', 'Sharpe Ratio']
    
    detail = ''
    for i in range(len(self.assets)):
      name = self.assets[i].get_name()
      if self.name == "GMV":
        self.ratios = self.gmv_weights
      percentage = int(self.ratios[i] * 100)

      detail += f'{name} ({percentage}%, {self.leverages[i]}x) '

    self.summary = [detail, self.backtest_result_df['Total Balance'][0], self.backtest_result_df['Total Balance'][-1],
                    str(round(self.backtest_result_df['Total CAGR'][-1], 2))+'%', str(round(self.backtest_result_df['Total MDD'][-1], 2))+'%',
                    round(self.backtest_result_df['Total St. Dev'][-1], 2), round(self.backtest_result_df['Total Sharpe'][-1], 2)]
    
    return self.summary

In [8]:
class Visualize():
  def __init__(self, portfolios):
    self.portfolios = portfolios

     # Assert all the dates for assets are equal and set portfolio start, end dates
    dates = [each.get_date() for each in self.portfolios]
    lendates = [len(each) for each in dates]
    assert len(set(lendates)) == 1
    self.date = dates[0]

  def line_plot(self, result, title, return_type):
#     for i in range(len(self.portfolios)):
#       plt.plot(self.date, self.portfolios[i].get_backtest_result().loc[:, f'Total {result}'], label=self.portfolios[i].get_name())
    
#     plt.title(title)
#     plt.legend(loc='best')
#     plt.yscale(return_type)
#     plt.show()
    
    data_list = []
    colm_list = []
    for i in range(len(self.portfolios)):
        data_list.append(self.portfolios[i].get_backtest_result().loc[:, f'Total {result}'])
        colm_list.append(self.portfolios[i].get_name())
    plot_df = pd.DataFrame(data_list).T
    plot_df.columns = colm_list
    plot_df.iplot(kind='line')
    
    
    
  def bar_plot(self, title, period):
#     for i in range(len(self.portfolios)):
#       if period == 'annual':
#         result = self.portfolios[i].periodic_result('annual')
#         plt.bar(result.index, result.loc[:, 'Total Annual Return'], label=self.portfolios[i].get_name())
#       elif period == 'monthly':
#         result = self.portfolios[i].periodic_result('monthly')
#         plt.bar(result.index, result.loc[:, 'Total Monthly Return'], label=self.portfolios[i].get_name())
    
#     plt.title(title)
#     plt.legend(loc='best')
#     plt.show()
    
    data_list = []
    colm_list = []
    for i in range(len(self.portfolios)):
      if period == 'annual':
        data_list.append(self.portfolios[i].periodic_result('annual').loc[:, 'Total Annual Return'])
        colm_list.append(self.portfolios[i].get_name())
      elif period == 'monthly':
        data_list.append(self.portfolios[i].periodic_result('monthly').loc[:, 'Total Monthly Return'])
        colm_list.append(self.portfolios[i].get_name())
    
    plot_df = pd.DataFrame(data_list).T
    plot_df.columns = colm_list
#     plot_df.plot.bar()
    plot_df.iplot(kind='bar', barmode='stack')

In [9]:
#class portfolio takes matrix of returns for every stock, and array of their weights
class GMVPortfolio:
    def __init__(self, mean_returns, cov_matrix, num_portfolios, risk_free_rate):
        self.mean_returns = mean_returns
        self.cov_matrix = cov_matrix
        self.num_portfolios = num_portfolios
        self.risk_free_rate = risk_free_rate

    def neg_sharpe_ratio(self, weights, mean_returns, cov_matrix, risk_free_rate):
        p_var, p_ret = self.portfolio_annualised_performance(weights, mean_returns, cov_matrix)
        return -(p_ret - risk_free_rate) / p_var

    def max_sharpe_ratio(self):
        num_assets = len(self.mean_returns)
        args = (self.mean_returns, self.cov_matrix, self.risk_free_rate)
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bound = (0.0,1.0)
        bounds = tuple(bound for asset in range(num_assets))
        result = optimize.minimize(self.neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                            method='SLSQP', bounds=bounds, constraints=constraints)
        return result
    
    def portfolio_volatility(self, weights, mean_returns, cov_matrix):
        return self.portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

    def min_variance(self):
        num_assets = len(self.mean_returns)
        args = (self.mean_returns, self.cov_matrix)
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bound = (0.0,1.0)
        bounds = tuple(bound for asset in range(num_assets))

        result = optimize.minimize(self.portfolio_volatility, num_assets*[1./num_assets,], args=args,
                            method='SLSQP', bounds=bounds, constraints=constraints)

        return result

    def portfolio_return(self, weights):
        return self.portfolio_annualised_performance(weights, self.mean_returns, self.cov_matrix)[1]
    
    def efficient_return(self, target):
        num_assets = len(self.mean_returns)
        args = (self.mean_returns, self.cov_matrix)

        constraints = ({'type': 'eq', 'fun': lambda x: self.portfolio_return(x) - target},
                       {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0,1) for asset in range(num_assets))
        result = optimize.minimize(self.portfolio_volatility, num_assets*[1./num_assets,], args=args, method='SLSQP', bounds=bounds, constraints=constraints)
        return result


    def efficient_frontier(self, returns_range):
        efficients = []
        for ret in returns_range:
            efficients.append(self.efficient_return(ret))
        return efficients
    
    def portfolio_annualised_performance(self, weights, mean_returns, cov_matrix):
        returns = np.sum(self.mean_returns*weights ) *252
        std = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix, weights))) * np.sqrt(252)
        return std, returns
    
    def random_portfolios(self):
        results = np.zeros((3, self.num_portfolios))
        weights_record = []
        for i in range(self.num_portfolios):
            weights = np.random.random(self.mean_returns.shape[0])
            weights /= np.sum(weights)
            weights_record.append(weights)
            portfolio_std_dev, portfolio_return = self.portfolio_annualised_performance(weights, self.mean_returns, self.cov_matrix)
            results[0,i] = portfolio_std_dev
            results[1,i] = portfolio_return
            results[2,i] = (portfolio_return - self.risk_free_rate) / portfolio_std_dev
        return results, weights_record       
    
    def display_calculated_ef_with_random(self):
        results, _ = self.random_portfolios()

        max_sharpe = self.max_sharpe_ratio()
        sdp, rp = self.portfolio_annualised_performance(max_sharpe['x'], self.mean_returns, self.cov_matrix)
        max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=df.columns,columns=['allocation'])
        max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
        max_sharpe_allocation = max_sharpe_allocation.T

        min_vol = self.min_variance()
        sdp_min, rp_min = self.portfolio_annualised_performance(min_vol['x'], self.mean_returns, self.cov_matrix)
        min_vol_allocation = pd.DataFrame(min_vol.x,index=df.columns,columns=['allocation'])
        min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
        min_vol_allocation = min_vol_allocation.T

        print("-"*80)
        print("Maximum Sharpe Ratio Portfolio Allocation\n")
        print("Annualised Return:", round(rp,2))
        print("Annualised Volatility:", round(sdp,2))
        print("\n")
        print(max_sharpe_allocation)
        print("-"*80)
        print("Minimum Volatility Portfolio Allocation\n")
        print("Annualised Return:", round(rp_min,2))
        print("Annualised Volatility:", round(sdp_min,2))
        print("\n")
        print(min_vol_allocation)

        plt.figure(figsize=(10, 7))
        plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
        plt.colorbar()
        plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
        plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

        target = np.linspace(rp_min, 0.32, 50)
        efficient_portfolios = self.efficient_frontier(target)
        plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
        plt.title('Calculated Portfolio Optimization based on Efficient Frontier')
        plt.xlabel('annualised volatility')
        plt.ylabel('annualised returns')
        plt.legend(labelspacing=0.8)
        
    def display_ef_with_selected(self):
        max_sharpe = self.max_sharpe_ratio()
        sdp, rp = self.portfolio_annualised_performance(max_sharpe['x'], self.mean_returns, self.cov_matrix)
        max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=df.columns,columns=['allocation'])
        max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
        max_sharpe_allocation = max_sharpe_allocation.T

        min_vol = self.min_variance()
        sdp_min, rp_min = self.portfolio_annualised_performance(min_vol['x'], self.mean_returns, self.cov_matrix)
        min_vol_allocation = pd.DataFrame(min_vol.x,index=df.columns,columns=['allocation'])
        min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
        min_vol_allocation = min_vol_allocation.T

        an_vol = np.std(returns) * np.sqrt(252)
        an_rt = self.mean_returns * 252

        print("-"*80)
        print("Maximum Sharpe Ratio Portfolio Allocation\n")
        print("Annualised Return:", round(rp,2))
        print("Annualised Volatility:", round(sdp,2))
        print("\n")
        print(max_sharpe_allocation)
        print("-"*80)
        print("Minimum Volatility Portfolio Allocation\n")
        print("Annualised Return:", round(rp_min,2))
        print("Annualised Volatility:", round(sdp_min,2))
        print("\n")
        print(min_vol_allocation)
        print("-"*80)
        print("Individual Stock Returns and Volatility\n")
        for i, txt in enumerate(df.columns):
            print(txt,":","annuaised return",round(an_rt[i],2),", annualised volatility:",round(an_vol[i],2))
        print("-"*80)

        fig, ax = plt.subplots(figsize=(10, 7))
        ax.scatter(an_vol,an_rt,marker='o',s=200)

        for i, txt in enumerate(df.columns):
            ax.annotate(txt, (an_vol[i],an_rt[i]), xytext=(10,0), textcoords='offset points')
        ax.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
        ax.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

        target = np.linspace(rp_min, 0.34, 50)
        efficient_portfolios = self.efficient_frontier(target)
        ax.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
        ax.set_title('Portfolio Optimization with Individual Stocks')
        ax.set_xlabel('annualised volatility')
        ax.set_ylabel('annualised returns')
        ax.legend(labelspacing=0.8)
        
    def display_ef_with_selected_and_random(self):
        results, _ = self.random_portfolios()
        
        max_sharpe = self.max_sharpe_ratio()
        sdp, rp = self.portfolio_annualised_performance(max_sharpe['x'], self.mean_returns, self.cov_matrix)
        max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=df.columns,columns=['allocation'])
        max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
        max_sharpe_allocation = max_sharpe_allocation.T

        min_vol = self.min_variance()
        sdp_min, rp_min = self.portfolio_annualised_performance(min_vol['x'], self.mean_returns, self.cov_matrix)
        min_vol_allocation = pd.DataFrame(min_vol.x,index=df.columns,columns=['allocation'])
        min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
        min_vol_allocation = min_vol_allocation.T

        an_vol = np.std(returns) * np.sqrt(252)
        an_rt = self.mean_returns * 252

        print("-"*80)
        print("Maximum Sharpe Ratio Portfolio Allocation\n")
        print("Annualised Return:", round(rp,2))
        print("Annualised Volatility:", round(sdp,2))
        print("\n")
        print(max_sharpe_allocation)
        print("-"*80)
        print("Minimum Volatility Portfolio Allocation\n")
        print("Annualised Return:", round(rp_min,2))
        print("Annualised Volatility:", round(sdp_min,2))
        print("\n")
        print(min_vol_allocation)
        print("-"*80)
        print("Individual Stock Returns and Volatility\n")
        for i, txt in enumerate(df.columns):
            print(txt,":","annuaised return",round(an_rt[i],2),", annualised volatility:",round(an_vol[i],2))
        print("-"*80)

        fig, ax = plt.subplots(figsize=(10, 7))
        ax.scatter(an_vol,an_rt,marker='o',s=200)
        ax.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)

        for i, txt in enumerate(df.columns):
            ax.annotate(txt, (an_vol[i],an_rt[i]), xytext=(10,0), textcoords='offset points')
        ax.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
        ax.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

        target = np.linspace(rp_min, 0.34, 50)
        efficient_portfolios = self.efficient_frontier(target)
        ax.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
        ax.set_title('Portfolio Optimization with Individual Stocks')
        ax.set_xlabel('annualised volatility')
        ax.set_ylabel('annualised returns')
        ax.legend(labelspacing=0.8)        

## 2. GMV Portfolio

### 1) GMV - Calculation

In [10]:
gmv_portfolio = ['AMZN', 'AAPL', 'GOOGL', 'SBUX', 'SO', 'INTC', 'CSCO', 'MET']
gmv_asset = GMV_Asset(gmv_portfolio, '2016-01-01', '2020-10-31')
gmv_asset.create_df()

In [11]:
GMV = GMVPortfolio(gmv_asset.get_mean_returns(), gmv_asset.get_cov_matrix(), num_portfolios=25000, risk_free_rate=0.0178)
gmv_weights = [round(i, 2)for i in GMV.min_variance().x]
gmv_asset.get_non_zero_weights(gmv_weights)

In [12]:
gmv_weights

[0.18, 0.0, 0.12, 0.15, 0.43, 0.0, 0.12, 0.0]

In [13]:
gmv_portfolio = Portfolio('GMV', gmv_asset.non_zero_assets, gmv_asset.non_zero_weights, [1 for i in gmv_portfolio], 1000, 1)
gmv_backtest_df = gmv_portfolio.backtest()
backtest_result_df = gmv_portfolio.backtest_result()
gmv_annual_return_df = gmv_portfolio.periodic_result('annual')
gmv_monthly_return_df = gmv_portfolio.periodic_result('monthly')
gmv_portfolio.get_summary()

EUM100:                     0         1         2         3         4
Date                                                        
2016-01-04       NaN       NaN       NaN       NaN       NaN
2016-01-05 -0.005024  0.002752  0.006694  0.007017 -0.004544
2016-01-06 -0.001799 -0.002889 -0.008866  0.000211 -0.010650
2016-01-07 -0.039058 -0.024140 -0.024772 -0.008444 -0.023068
2016-01-08 -0.001464 -0.013617 -0.001058 -0.007451 -0.024794
2016-01-11  0.017610  0.002955  0.021013  0.004719  0.019774
2016-01-12  0.000243  0.016738  0.028364 -0.008326  0.003166
2016-01-13 -0.058392 -0.034575 -0.026741  0.007535 -0.029586
2016-01-14  0.019233  0.016426  0.019181  0.007265  0.002439
2016-01-15 -0.038482 -0.028576 -0.016616  0.000636 -0.042174
2016-01-19  0.007541  0.012090  0.009483  0.016324  0.009738
2016-01-20 -0.004717 -0.000723 -0.027839 -0.016270 -0.039832
2016-01-21  0.005684  0.011286  0.037070 -0.007421  0.000000
2016-01-22  0.037147  0.025858  0.002372  0.011536  0.020524
2016-01-25  0.0

['AMZN (19%, 1x) GOOGL (12%, 1x) SBUX (12%, 1x) SO (39%, 1x) CSCO (19%, 1x) ',
 1000.0,
 2120.1971401583705,
 '16.86%',
 '-29.49%',
 1.23,
 0.9]

### 2) GMV - Efficient Frontier

In [14]:
# GMV.display_calculated_ef_with_random()
# GMV.display_ef_with_selected()
GMV.display_ef_with_selected_and_random()

NameError: ignored

## 3. 60/40 Portfolio

In [ ]:
asset1 = Asset('^IXIC', 'Asset 1', '2016-01-01', '2020-10-31')
asset2 = Asset('VUSTX', 'Asset 2', '2016-01-01', '2020-10-31')

sixty_forty_portfolio = Portfolio('60/40', [asset1, asset2], [6, 4], [1, 1], 1000, 3)
sixty_forty_backtest_df = sixty_forty_portfolio.backtest()
sixty_forty_backtest_result_df = sixty_forty_portfolio.backtest_result()
sixty_forty_annual_return_df = sixty_forty_portfolio.periodic_result('annual')
sixty_forty_monthly_return_df = sixty_forty_portfolio.periodic_result('monthly')
sixty_forty_summary = sixty_forty_portfolio.get_summary()

EUM1 : 1013.0771185189658
EUM2 : 0.6
EUM3 : 1.0001150185323233
EUM1 : 1013.0771185189658
EUM2 : 0.4
EUM3 : 1.008383548272043
EUM1 : 1030.1979352903948
EUM2 : 0.6
EUM3 : 1.0132698481717843
EUM1 : 1030.1979352903948
EUM2 : 0.4
EUM3 : 1.0049199204104486
EUM1 : 1096.4100387432372
EUM2 : 0.6
EUM3 : 1.008132260127424
EUM1 : 1096.4100387432372
EUM2 : 0.4
EUM3 : 0.9946505827494936
EUM1 : 1059.5700139552075
EUM2 : 0.6
EUM3 : 0.9909851037131053
EUM1 : 1059.5700139552075
EUM2 : 0.4
EUM3 : 1.0058259046294769
EUM1 : 1124.5610921285013
EUM2 : 0.6
EUM3 : 1.0028469516356022
EUM1 : 1124.5610921285013
EUM2 : 0.4
EUM3 : 0.9924431577229889
EUM1 : 1170.1170465121581
EUM2 : 0.6
EUM3 : 0.999360359400297
EUM1 : 1170.1170465121581
EUM2 : 0.4
EUM3 : 0.9989387282880854
EUM1 : 1211.3598150582675
EUM2 : 0.6
EUM3 : 1.003195871592665
EUM1 : 1211.3598150582675
EUM2 : 0.4
EUM3 : 0.9983608274734843
EUM1 : 1268.6760274865424
EUM2 : 0.6
EUM3 : 1.0149940483697186
EUM1 : 1268.6760274865424
EUM2 : 0.4
EUM3 : 0.9887188048040

In [ ]:
sixty_forty_summary

['Asset 1 (60%, 1x) Asset 2 (40%, 1x) ',
 1000.0,
 1943.2611274026767,
 '14.73%',
 '-17.18%',
 0.73,
 1.24]

## 4. All Weather Portfolio

In [ ]:
asset3 = Asset('VTI', 'Asset 3', '2016-01-01', '2020-10-31')
asset4 = Asset('EDV', 'Asset 4', '2016-01-01', '2020-10-31')
asset5 = Asset('IEI', 'Asset 5', '2016-01-01', '2020-10-31')
asset6 = Asset('GLD', 'Asset 6', '2016-01-01', '2020-10-31')
asset7 = Asset('GSG', 'Asset 7', '2016-01-01', '2020-10-31')

awf_portfolio = Portfolio('All Weather', [asset3, asset4, asset5, asset6, asset7], [3, 4, 1.5, 0.75, 0.75], [1, 1, 1, 1, 1], 1000, 12)
awf_backtest_df = awf_portfolio.backtest()
awf_backtest_result_df = awf_portfolio.backtest_result()
awf_annual_return_df = awf_portfolio.periodic_result('annual')
awf_monthly_return_df = awf_portfolio.periodic_result('monthly')
awf_summary = awf_portfolio.get_summary()

Portfolio Backtest Complete
Total Complete
Asset 3 Complete
Asset 4 Complete
Asset 5 Complete
Asset 6 Complete
Asset 7 Complete
Annual Result Complete
Monthly Result Complete


In [ ]:
awf_summary

['Asset 3 (30%, 1x) Asset 4 (40%, 1x) Asset 5 (15%, 1x) Asset 6 (7%, 1x) Asset 7 (7%, 1x) ',
 1000.0,
 1527.8005406978286,
 '9.16%',
 '-14.58%',
 0.52,
 1.1]

## 5. Result Visualization

In [ ]:
aw_64 = Visualize([gmv_portfolio, sixty_forty_portfolio, awf_portfolio])

In [ ]:
aw_64.line_plot('Drawdown', 'GMV VS 60/40 VS All Weather Drawdown', 'linear')

In [ ]:
aw_64.bar_plot('GMV VS 60/40 VS All Weather Monthly Return', 'monthly')

Monthly Result Complete
Monthly Result Complete
Monthly Result Complete
